Establish project and weights directories

In [ ]:
import sys
import os

# Manually set the project root directory (adjust if needed)
project_root = os.path.abspath("..")  # Moves up one level to project root
# Add the project directory to sys.path
sys.path.append(project_root)

# Ensure the directory exists
model_save_path = os.path.join(project_root, 'Models', 'Weights', 'GRU')
os.makedirs(model_save_path, exist_ok=True)  # Creates directory if it doesn't exist

Load and preprocess data & Train the Model with Optuna

In [ ]:
import torch
from Models.GRU import GRUModel   # Import the GRU model
# Import modularized functions
from Helper.dataPreprocessing import load_data, prepare_dataloader, TRAIN_DATA_PATH_1990S
from Helper.PyTorchModular import optuna_tune_and_train

# Load and preprocess data
config = {"use_fft": False, "use_exog": False}

(X_train_seq, y_train_seq, X_valid_seq, y_valid_seq, X_test_seq, y_test_seq, observation_dates, y_scaler) = load_data(
    TRAIN_DATA_PATH_1990S, sequence_length=12, config=config
)

X_train_seq = X_train_seq.reshape(X_train_seq.shape[0], X_train_seq.shape[1], 1)
X_valid_seq = X_valid_seq.reshape(X_valid_seq.shape[0], X_valid_seq.shape[1], 1)

y_train_seq = y_train_seq.reshape(-1, 1)
y_valid_seq = y_valid_seq.reshape(-1, 1)

batch_size = 32
train_loader = prepare_dataloader(X_train_seq, y_train_seq, batch_size=batch_size)
val_loader = prepare_dataloader(X_valid_seq, y_valid_seq, batch_size=batch_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Run Optuna & Train Model
model, metadata = optuna_tune_and_train(
    model_class=GRUModel, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    device=device,
    max_epochs=50,
    model_save_path=model_save_path,
    model_name="GRU_inflation",
    use_best_hyperparams=False,  # Set to False to force Optuna tuning every time
    n_trials=20,
    verbose=True
)


Make Predictions on Validation Set

In [ ]:
from Evaluation.Helper.evaluation_helpers import evaluate_model

# Evaluate Model
df_comparison, rmse = evaluate_model(
    model=model, 
    val_loader=val_loader, 
    y_scaler=y_scaler, 
    observation_dates=observation_dates, 
    device=device,
    verbose=True
)